In [4]:
def create_permutation_matrix(s):
    s = list(s)
    def key(si):
        return si[0]
    s_p = [(si,i) for i,si in enumerate(s)]
    s_p.sort(key = key)
    l = len(s)
    M = MatrixSpace(ZZ,l)
    P = M([[0]*l]*l)
    for i in range(l):
        P[i,s_p[i][1]] = 1
    return P

In [234]:
def Basis(F0,s = None):
    m,n = F0.dimensions()
    if (s == None):
        s = [0]*m

    pi = create_permutation_matrix(s)
    P,L,U = (pi*F0).LU()
    r = U.rank()
    Lr = L[:r,:r]
    G = L[r:,:r]
    K = F0.base_ring()
    pR.<x> = K[]
    Pinv = P.inverse_of_unit()
    M = block_matrix(2,2,[[x,0],[-G*Lr.inverse(), 1]])*Pinv*pi
    MS = MatrixSpace(ZZ,m)
    u = MS(Pinv)*pi*vector(s) + vector([1]*r+[0]*(m-r))
    return M,u

In [235]:
low_bound = -4
high_bound = 5
m = 4
n = 2

In [236]:
s = [randint(low_bound, high_bound) for i in range(m)]

In [249]:
pR.<x> = GF(97)[]

In [250]:
M = MatrixSpace(pR,m,n)

In [251]:
A = M.random_element()

In [252]:
B = Basis(A)[0]

In [253]:
B.is_minimal_approximant_basis(A,1)

False

In [254]:
MB,u = Basis(A)

In [259]:
def M_Basis(F,sigma,shift = None):
    pR = F.base_ring()
    x = pR.gen()
    m,n = F.dimensions()
    R = parent(F)
    MS = MatrixSpace(pR,m)
    if (shift == None):
        shift = [0]*m
    P,u = Basis(F.coefficient_matrix(0), shift)
    for k in range(1,sigma):
        F_prime = R(x^(-k)*MS(P)*F).coefficient_matrix(0)
        M,u = Basis(F_prime, u)
        P = M*P
    return P,u

In [270]:
M,u = M_Basis(A,3)

In [283]:
M.is_minimal_approximant_basis(A,3)

False

In [286]:
M.is_reduced()

True